In [1]:
# testing datadase_utils file

In [20]:
import sys
import yaml

# sql connection
import mysql.connector as mysql
from mysql.connector import Error
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

In [11]:
database_type_flag = 'local'

In [12]:
valid_database_types = ['local', 'aws','gcp']
if database_type_flag not in valid_database_types:
    print(f"Invalid database_type_flag. Expected one of: {valid_database_types}")

In [13]:
try:
    with open('./config/credentials.yml') as f:
        credentails_data = yaml.safe_load(f)
except Exception as e:
    print(f'Failed to open credentials file: {e}')
    sys.exit(1)

In [14]:
credentials_data

{'aws_database': {'hostname': 'aba.crvkycawugwh.us-east-1.rds.amazonaws.com',
  'port': '3306',
  'username': 'franklin_ajisogun',
  'password': 'Water1'},
 'gcp_database': {'hostname': '35.236.242.49',
  'port': '3306',
  'username': 'root',
  'password': 'applesandoranges'},
 'local_database': {'hostname': 'localhost',
  'port': '3306',
  'username': 'root',
  'password': 'test@123'},
 'email_detail': {'email': 'franklintest03@gmail.com',
  'password': 'qvjtgnavwhqlpbbw'}}

In [32]:
credentials = credentails_data.get(f'{database_type_flag}_database')
credentials

{'hostname': 'localhost',
 'port': 3306,
 'username': 'root',
 'password': 'test@123'}

In [33]:
if credentials.get('port') is not None:
    try:
        credentials['port'] = int(credentails['port'])
    except ValueError:
        print("Error: Port number must be an integer") 
        sys.exit(1)

In [34]:
host_input, user_input, password_input,port_input = credentials["hostname"], credentials["username"],\
                                                    credentials["password"],credentials["port"]


In [37]:
credentials_info = {"host": host_input,
                    "user": user_input,
                    "password": password_input,
                    "port": port_input}

try:
    connection = mysql.connect(**credentials_info)
    if connection.is_connected():
        print("Connected to MySQL database...")
except mysql.Error as e:
    print(f"Failed to connect to MySQL database: {e}")

Connected to MySQL database...


In [38]:
import database_utils

In [39]:
credentials = database_utils.yaml_credentials(database_type_flag) 
connection = database_utils.connect_db(credentials)

INFO:database_utils:Connected to MySQL database...


In [40]:
connection

In [41]:
cursor = connection.cursor()

In [42]:
cursor.execute("SHOW DATABASES")

In [43]:
for db in cursor.fetchall():
    print(db)

('customer_db',)
('information_schema',)
('iris_db',)
('mysql',)
('performance_schema',)
('sys',)


In [ ]:
def load_data_in_database(df, database_type_flag = "local", db_name = 'covid_db',  table_name = 'covid'):

    # uppercase columns
    # df.columns = ['CASES_DATE', 'COUNTY', 'STATE', 'FIPS', 'CASES', 'DEATHS', 'DB_DATE_UPLOAD']

    credentials = database_utils.yaml_credentials(database_type_flag) 
    connection = database_utils.connect_db(credentials)

    if connection is not None:
        database_utils.ensure_database_exists(connection, db_name)

        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {db_name}.{table_name} (
             CASES_DATE DATE,
             COUNTY varchar(255),
             STATE varchar(255),
             FIPS  INT,
             CASES INT,
             DEATHS INT,
             DB_DATE_UPLOAD DATE NOT NULL, 
             SOURCE_TYPE varchar(255)
        """
        database_utils.create_table(connection, create_table_query, db_name, table_name)
        #database_utils.load_data_to_table(df, credentials, db_name, table_name, chunksize=10000)
        connection.close()